Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [ ]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
# from __future__ import print_function
import os
import copy
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
import gzip
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [ ]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

In [ ]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

def read_word_reverse(filename):
    with gzip.open(filename, 'rb') as f:
        return tf.compat.as_str(f.read())
    
text = read_data(filename)

print('Data size %d' % len(text))

In [ ]:
def write_word_reverse(text, filename):
    if os.path.exists(filename):
        print('Found {}.'.format(filename))
        return
    text_size = len(text)
    cursor = 0

    with gzip.open(filename, 'wb') as f:
        while cursor < text_size:
            if text[cursor] == ' ':
                f.write(text[cursor].encode())
                cursor += 1
            else:
                try:
                    end = text.index(' ', cursor)                
                except ValueError:
                    end = text_size
                chars = list(text[cursor:end])
                chars.reverse()
                word = ''.join(chars)
                f.write(word.encode())
                cursor = end
        print('Create {}.'.format(filename))

# maybe write
write_word_reverse(text, 'reverse.txt.gz')

In [ ]:
text_reversed = read_word_reverse('reverse.txt.gz')
print('Data size {}'.format(len(text_reversed)))

Create a small validation set.

In [ ]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

Utility functions to map characters to vocabulary IDs and back.

In [ ]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Function to generate a training batch for the LSTM model.

In [ ]:
def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

# Unigram with embedding

For bigram implementation, please refer to *6_lstm-bigram*.

In [ ]:
batch_size=64
num_unrollings=10

class BatchIdGenerator(object):
    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [ offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()
    
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=self._batch_size, dtype=np.int32)
        for b in range(self._batch_size):
            batch[b] = char2id(self._text[self._cursor[b]])
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
    
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(ids):
    """Convert a character id into the character"""
    return [id2char(c) for c in ids]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

def sample_id(prediction):
    """Turn a (column) prediction into one character id."""
    return sample_distribution(prediction[0])

def logprob(predictions, label_ids):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    labels = np.zeros((len(label_ids), vocabulary_size), dtype=np.float32)
    for i in range(len(label_ids)):
        labels[i, label_ids[i]] = 1.0
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

train_batches = BatchIdGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchIdGenerator(valid_text, 1, 1)
train_label_batches = BatchIdGenerator(text_reversed[valid_size:], batch_size, num_unrollings)
valid_label_batches = BatchIdGenerator(text_reversed[:valid_size], 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(train_label_batches.next()))
print(batches2string(valid_label_batches.next()))

In [ ]:
# Create embedding lookup parameters

embedding_params = np.zeros(shape=(vocabulary_size, vocabulary_size), dtype=np.float32)
for c in (string.ascii_lowercase + ' '):
    cid = char2id(c)
    embedding_params[cid, cid] = 1.0
    
# print(embedding_params)

# Simple LSTM Model.

In [ ]:
num_nodes = 64
keep_prob = 0.5

graph = tf.Graph()
with graph.as_default():
    
    # Parameters:    
    # Input variable
    ifcox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes * 4], -0.1, 0.1))
    # Memory cell
    ifcom = tf.Variable(tf.truncated_normal([num_nodes, num_nodes * 4], -0.1, 0.1))
    # Bias
    ifcob = tf.Variable(tf.zeros([1, num_nodes * 4]))

    # Variables saving state across unrollings.
    saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))
    
    # Definition of the cell computation.
    def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""

        all_gate = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
        input_gate, forget_gate, update, output_gate = tf.split(1, 4, all_gate)
        input_gate = tf.sigmoid(input_gate)
        forget_gate = tf.sigmoid(forget_gate)
        output_gate = tf.sigmoid(output_gate)
#         state = forget_gate * tf.nn.dropout(state, keep_prob) + input_gate * tf.tanh(cell_gate)
        state = forget_gate * state + input_gate * tf.tanh(update)
        return output_gate * tf.tanh(state), state

    # Input data.
    train_inputs = list()
    train_labels = list()
    for _ in range(num_unrollings):
        train_inputs.append(tf.placeholder(tf.int32, shape=[batch_size]))
        train_labels.append(tf.placeholder(tf.int32, shape=[batch_size]))

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        # embeddings...
        embedded = tf.nn.embedding_lookup(embedding_params, i)
        output, state = lstm_cell(embedded, output, state)
        outputs.append(output)

    train_label_embeddes = [tf.nn.embedding_lookup(embedding_params, l) for l in train_labels]

    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_label_embeddes)))

    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)
    
    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.int32, shape=[1])
    sample_input_embedded = tf.nn.embedding_lookup(embedding_params, sample_input)
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input_embedded, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                    saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [ ]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        train = train_batches.next()
        label = train_label_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings):
            feed_dict[train_inputs[i]] = train[i]
            feed_dict[train_labels[i]] = label[i]
        _, l, predictions, lr = session.run(
          [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few batches.
            print(
              'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            labels = np.concatenate(label[:num_unrollings])
            print('Minibatch perplexity: %.2f' % float(
                np.exp(logprob(predictions, labels))))
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample_id(random_distribution())
                    sentence = characters([feed])[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval({sample_input: [feed]})
                        feed = sample_id(prediction)
                        sentence += characters([feed])[0]
                    print(sentence)
                print('=' * 80)
            # Measure validation set perplexity.
            reset_sample_state.run()
            valid_logprob = 0
            for _ in range(valid_size):
                b = valid_batches.next()
                c = valid_label_batches.next()
#                 print('b -> c: {}, {}'.format(characters(b[0]), characters(c[0])))
                predictions = sample_prediction.eval({sample_input: b[0]})
                valid_logprob = valid_logprob + logprob(predictions, c[0])
            print('Validation set perplexity: %.2f' % float(np.exp(
                valid_logprob / valid_size)))